# Development Notebook for Cortical Crowding Project

## Dependencies

In [ ]:
import os, sys
from pathlib import Path

import numpy as np
import pandas as pd
import neuropythy as ny
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.optimize
from scipy.stats import gmean
from scipy.optimize import curve_fit

In [ ]:
#mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.family'] = 'HelveticaNeue'
mpl.rcParams['font.size'] = 10
mpl.rcParams['font.weight'] = 'light'
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['figure.dpi'] = 576  # 72*8
mpl.rcParams['hatch.color'] = 'white'

In [ ]:
# We want to be able to load in libraries that are in this repository's src directory,
# so we add src to the system path:
try:
    import corticalcrodwing as cc
except ModuleNotFoundError:
    # This probably happens because the corticalcrowding library hasn't been
    # installed yet; we can add the src directory to the path to work around
    # this here.
    sys.path.append('../src')
    # Now we can import corticalcrowding from the src directory:
    import corticalcrowding as cc

## Configuration

In [ ]:
# The root path where data is stored:
data_path = Path('/data/crowding')

# The crowding data CSV file:
crowding_data_filename = data_path / 'crowding_data_withID.csv'

In [ ]:
# The list of subjects:
sids_NYU = [
    'sub-wlsubj070',
    'sub-wlsubj114',
    'sub-wlsubj121',
    'sub-wlsubj135']

# 36 is used
sids_NEI = [ 
    'sub-wlsubj119',
    'sub-wlsubj127',
    'sub-wlsubj136',
    'sub-wlsubj137',
    'sub-wlsubj143',
    'sub-wlsubj144',
    'sub-wlsubj145',
    'sub-wlsubj146',
    'sub-wlsubj147',
    'sub-wlsubj148',
    'sub-wlsubj149',
    'sub-wlsubj150',
    'sub-wlsubj151',
    'sub-wlsubj152',
    'sub-wlsubj153',
    'sub-wlsubj154',
    'sub-wlsubj155',
    'sub-wlsubj156',
    'sub-wlsubj157',
    'sub-wlsubj158',
    'sub-wlsubj159',
    'sub-wlsubj160',
    'sub-wlsubj161',
    'sub-wlsubj162',
    'sub-wlsubj163',
    'sub-wlsubj164',
    'sub-wlsubj165',
    'sub-wlsubj166',
    'sub-wlsubj167',
    'sub-wlsubj168',
    'sub-wlsubj170',
    'sub-wlsubj171',
    'sub-wlsubj172',
    'sub-wlsubj173',
    'sub-wlsubj174',
    'sub-wlsubj175',
    'sub-wlsubj176']

sids_orig = sids_NYU + sids_NEI

## Noah's explorations of the crowding data (2025-10-07)

In [ ]:
crowding_data = pd.read_csv(crowding_data_filename)

crowddat = crowding_data.rename(
    columns=dict(
        Eccen_X='x', Eccen_Y='y',
        CrowdingDistance='crowding_distance',
        RadialEccentricity='eccentricity',
        FlankinDirection='flankers',
        ID='sid',
        Session='session'))
crowddat['hemi'] = np.where(crowddat['x'] < 0, 'rh', 'lh')
crowddat.loc[crowddat['x'].values == 0, 'hemi'] = 'lr'
theta = np.arctan2(crowddat['y'], crowddat['x'])
meridian_no = np.round(np.mod(theta * 180/np.pi + 360, 360) / 360 * 4).astype(int)
meridians = np.array(['right', 'upper', 'left', 'lower'])
crowddat['meridian'] = meridians[meridian_no]
crowddat = crowddat[
    ['sid', 'hemi', 'meridian', 'eccentricity', 'flankers', 'session',
     'x', 'y',
     'crowding_distance']]
crowddat

In [ ]:
df = dict(sid=[], hemi=[], label=[], a=[], b=[], loss=[])
for sid in sids_orig:
    print(sid)
    for h in ['lh','rh','lr']:
        for lbl in [1,2,3,4]:
            try:
                r = cc.cmag.fit_cumarea(sid, h, lbl)
            except Exception as e:
                print(f"  - Skipping: {type(e)}")
                continue
            df['sid'].append(sid)
            df['hemi'].append(h)
            df['label'].append(lbl)
            df['a'].append(r.x[0])
            df['b'].append(r.x[1])
            df['loss'].append(r.fun)
cmagdat = pd.DataFrame(df)

In [ ]:
dataframe = pd.merge(crowddat, cmagdat, on=('sid', 'hemi'))

cmag_areal = cc.cmag.HH91(
    dataframe['eccentricity'],
    dataframe['a'],
    dataframe['b'])
cmag_rad = np.sqrt(cmag_areal / 2)
cmag_tan = cmag_rad * 2
vmag_rad = 1 / cmag_rad
vmag_tan = 1 / cmag_tan

d_pred = np.empty_like(cmag_rad)
flankers = dataframe['flankers'].values
cmag_pred = np.where(flankers == 'radial', cmag_rad, cmag_tan)
vmag_pred = np.where(flankers == 'radial', vmag_rad, vmag_tan)

dataframe['cmag_are'] = cmag_areal
dataframe['cmag_rad'] = cmag_rad
dataframe['cmag_tan'] = cmag_tan
dataframe['vmag_rad'] = vmag_rad
dataframe['vmag_tan'] = vmag_tan
dataframe['cmag_lin'] = cmag_pred
dataframe['vmag_lin'] = vmag_pred

dataframe

In [ ]:
def crowddist_model(df, params):
    (d_hrz, d_upp, d_low) = (params[0], params[0], params[0])
    gains_sid = params[1:]
    (ii_upp, ii_low) = [
        df['meridian'] == k for k in ('upper', 'lower')]
    ii_hrz = ~(ii_upp | ii_low)
    (ii_hrz, ii_upp, ii_low) = [
        np.where(ii)[0] for ii in (ii_hrz, ii_upp, ii_low)]
    (ii_hrz, ii_upp, ii_low) = [
        ii[np.argsort(df['sid'].values[ii])]
        for ii in (ii_hrz, ii_upp, ii_low)]
    (vm_hrz, vm_upp, vm_low) = [
        df['vmag_lin'].values[ii]
        for ii in (ii_hrz, ii_upp, ii_low)]
    dpred_hrz = vm_hrz * d_hrz
    dpred_upp = vm_upp * d_upp
    dpred_low = vm_low * d_low
    if len(gains_sid) > 0:
        sii = np.empty(len(df), dtype=int)
        dfsids = df['sid'].values
        for (k,sid) in enumerate(np.unique(dfsids)):
            sii[dfsids == sid] = k
        dpred_hrz *= gains_sid[sii[ii_hrz]]
        dpred_upp *= gains_sid[sii[ii_upp]]
        dpred_low *= gains_sid[sii[ii_low]]
    dpred = np.empty_like(df['cmag_lin'].values)
    dpred[ii_hrz] = dpred_hrz
    dpred[ii_upp] = dpred_upp
    dpred[ii_low] = dpred_low
    return dpred
def crowddist_loss(data, label, params, tx=True):
    if tx:
        params = np.exp(params)
    df = data[data['label'] == label]
    dpred = crowddist_model(df, params)
    d = df['crowding_distance']
    diff = d - dpred
    return np.var(diff)

In [ ]:
def sigmoid(x, /, scale=1):
    """Returns the sigmoid function of the argument.
    
    The sigmoid function is defined as: ``sigmoid(x) = 1 / (1 + exp(-x))``.
    It is the inverse of the ``logit`` function; i.e., if ``p = sigmoid(x)``
    then ``x = logit(p)``.
    
    Parameters
    ----------
    x : number-like
        The argument (or arguments) to the sigmoid function.
    scale : float, optional
        The scale of the sigmoid (default: 1). ``sigmoid(x, s)`` is equivalent
        to ``sigmoid(x * s, 1)``.
    
    Returns
    -------
    number-like
        The sigmoid of `x`.
    """
    return 1/(1 + np.exp(-scale*x))
def logit(p, /, scale=1):
    """Returns the logit function of the argument.
    
    The logit function is defined as: ``logit(p) = log(p / (1 - p))``.
    It is the inverse of the ``sigmoid`` function; i.e., if ``p = sigmoid(x)``
    then ``x = logit(p)``.
    
    Parameters
    ----------
    p : number-like
        The argument (or arguments) to the logit function.
    scale : float, optional
        The scale of the logit (default: 1). ``logit(p, s)`` is equivalent
        to ``logit(p / s, 1)``.
    
    Returns
    -------
    number-like
        The logit of `p`.
    """
    return np.log(p / (1 - p)) / scale
class CrowdingModel:
    """A model of visual crowding that assumes a fixed crowding distance on the
    cortical surface.
    """
    __slots__ = (
        's',
        'g_upp',
        'g_low',
        'is_optform')
    def __init__(self, s=1.4, g_upp=9/16, g_low=15/16, optform=False):
        # Check the arguments:
        if isinstance(s, CrowdingModel):
            # Make a copy and ignore the other arguments.
            self.s = s.s
            self.g_upp = s.g_upp
            self.g_low = s.g_low
            self.is_optform = s.is_optform
            return
        s = float(s)
        if not optform and s <= 0:
            raise ValueError("s must me a float greater than 0")
        g_upp = float(g_upp)
        if not optform and (g_upp <= 1/3 or g_upp >= 5/3):
            raise ValueError("g_upp must be a float between 0.5 and 1.5")
        g_low = float(g_low)
        if not optform and (g_low <= 1/3 or g_low >= 5/3):
            raise ValueError("g_low must be a float between 0.5 and 1.5")
        # Set the member values:
        self.s = s
        self.g_upp = g_upp
        self.g_low = g_low
        # The normal init function doesn't create transformed arguments.
        self.is_optform = optform
    @staticmethod
    def _toform(is_inform, val, txfn):
        return val if is_inform else txfn(val)
    @staticmethod
    def _g_tx(g):
        return logit((3 * g - 1)/4)
    @staticmethod
    def _g_xt(g):
        return (4 * sigmoid(g) + 1) / 3
    @property
    def s_stdform(self):
        return CrowdingModel._toform(not self.is_optform, self.s, np.exp)
    @property
    def s_optform(self):
        return CrowdingModel._toform(self.is_optform, self.s, np.log)
    @property
    def g_upp_stdform(self):
        return CrowdingModel._toform(
            not self.is_optform, self.g_upp, CrowdingModel._g_xt)
    @property
    def g_upp_optform(self):
        return CrowdingModel._toform(
            self.is_optform, self.g_upp, CrowdingModel._g_tx)
    @property
    def g_low_stdform(self):
        return CrowdingModel._toform(
            not self.is_optform, self.g_low, CrowdingModel._g_xt)
    @property
    def g_low_optform(self):
        return CrowdingModel._toform(
            self.is_optform, self.g_low, CrowdingModel._g_tx)
    @property
    def params_stdform(self):
        if not self.is_optform:
            return (self.s, self.g_upp, self.g_low)
        return (self.s_stdform, self.g_upp_stdform, self.g_low_stdform)
    @property
    def params_optform(self):
        if self.is_optform:
            return (self.s, self.g_upp, self.g_low)
        return (self.s_optform, self.g_upp_optform, self.g_low_optform)
    @property
    def params(self):
        return self.params_optform if self.is_optform else self.params_stdform
    @property
    def g_hrz_stdform(self):
        (_, gu, gl) = self.params_stdform
        return 2 - (gu + gl)/2
    @property
    def g_hrz_optform(self):
        return logit(self.g_hrz_stdform - 0.5)
    @property
    def g_hrz(self):
        return self.g_hrz_optform if self.is_optform else self.g_hrz_stdform
    @property
    def hva(self):
        g_hrz = self.g_hrz_stdform
        g_upp = self.g_upp_stdform
        g_low = self.g_low_stdform
        g_vrt = (g_upp + g_low) / 2
        return 2 * (g_hrz - g_vrt) / (g_hrz + g_vrt)
    @property
    def vma(self):
        g_upp = self.g_upp_stdform
        g_low = self.g_low_stdform
        return 2 * (g_low - g_upp) / (g_low + g_upp)
    def to_optform(self):
        if self.is_optform:
            return type(self)(self)
        s_optform = np.log(self.s)
        g_upp_optform = logit(self.g_upp - 0.5)
        g_low_optform = logit(self.g_low - 0.5)
        return type(self)(s_optform, g_upp_optform, g_low_optform, True)
    def to_stdform(self):
        if not self.is_optform:
            return type(self)(self)
        s = np.exp(self.s)
        g_upp = 0.5 + sigmoid(self.g_upp)
        g_low = 0.5 + sigmoid(self.g_low)
        return type(self)(s, g_upp, g_low)
    def predict(self, *,
                dataframe=dataframe,
                vdst_meas_key='crowding_distance',
                cmag_meas_key='cmag_lin',
                meridian_key='meridian',
                cdst_tag='cdst',
                cmag_tag='cmag',
                vdst_tag='vdst',
                vmag_tag='vmag',
                hma_tag='hma',
                vma_tag='vma',
                meas_tag='meas',
                pred_tag='pred',
                **kwargs):
        # Make a copy of the dataframe.
        dataframe = pd.DataFrame(dataframe)
        for (k,v) in kwargs.items():
            dataframe = dataframe[dataframe[k] == v]
        # We need our parameters and derived parameters:
        (s_pred, g_upper, g_lower) = self.params_stdform
        g_horiz = self.g_hrz_stdform
        g_vert = (g_upper + g_lower) / 2
        # Get the measurements from the dataframe:
        vdst_meas = dataframe[vdst_meas_key]  # in deg
        cmag_meas = dataframe[cmag_meas_key]  # in mm/deg
        # The model actually predicts a slightly different cmag, based on the
        # part of the visual field represented.
        ii_upper = dataframe[meridian_key] == 'upper'
        ii_lower = dataframe[meridian_key] == 'lower'
        ii_horiz = ~(ii_upper | ii_lower)
        cmag_meas = np.array(cmag_meas)
        cmag_meas[ii_upper] *= g_upper
        cmag_meas[ii_lower] *= g_lower
        cmag_meas[ii_horiz] *= g_horiz
        cmag_pred = s_pred / vdst_meas
        # The model predicts the same cortical distance everywhere.
        dataframe[f'{cdst_tag}_{meas_tag}'] = cmag_meas * vdst_meas
        dataframe[f'{cdst_tag}_{pred_tag}'] = s_pred
        dataframe[f'{cmag_tag}_{meas_tag}'] = cmag_meas
        dataframe[f'{cmag_tag}_{pred_tag}'] = cmag_pred
        dataframe[f'{vdst_tag}_{meas_tag}'] = vdst_meas
        dataframe[f'{vdst_tag}_{pred_tag}'] = s_pred / cmag_meas
        dataframe[f'{vmag_tag}_{meas_tag}'] = 1/cmag_meas
        dataframe[f'{vmag_tag}_{pred_tag}'] = 1/cmag_pred
        dataframe[f'{hma_tag}_{pred_tag}'] = (g_horiz - g_vert) / g_horiz
        dataframe[f'{vma_tag}_{pred_tag}'] = (g_lower - g_upper) / g_upper
        return dataframe
    def loss(self, /, prop='cdst', *, dataframe=dataframe, **kwargs):
        if isinstance(prop, str):
            prop = (prop,)
        df = self.predict(dataframe=dataframe, **kwargs)
        meastag = kwargs.get('meas_tag', 'meas')
        predtag = kwargs.get('pred_tag', 'pred')
        l = 0
        for p in prop: 
            meas = df[f"{p}_{meastag}"]
            pred = df[f"{p}_{predtag}"]
            p_loss = np.mean((meas - pred)**2)
            l += p_loss
        return l
    def loss_opt(self, params, /,
                 prop='cdst', *,
                 dataframe=dataframe, **kwargs):
        # Params are assumed to be in optform.
        if not self.is_optform:
            self.is_optform = True
        (self.s, self.g_upp, self.g_low) = params
        return self.loss(prop=prop, dataframe=dataframe, **kwargs)

In [ ]:
from scipy.optimize import minimize
from functools import partial

prop = ('cmag')
fits = []
mdls = []
for lbl in (1,2,3,4):
    model = CrowdingModel()
    df = dataframe[dataframe['label'] == lbl]
    params = np.array(model.params_optform)
    fit = minimize(
        partial(model.loss_opt, prop=prop, label=lbl, dataframe=df),
        params)
    fit.x[:] = model.params_stdform
    fits.append(fit)
    mdls.append(model)
params = np.stack([fit.x for fit in fits])

In [ ]:
params

In [ ]:
[float(mdls[k-1].loss(prop=prop, label=k))
 for k in (1,2,3,4)]

In [ ]:
params

The basic model we're using:

```text
s: cortical crowding [mm]
v: visual crowding [deg]
m: cortical mag [mm/deg]

s / v = m

S (predicted s)
V (predicted v)
M (predicted m)

min (S - mv)  ==> hV4 is best!
min (V - s/m) ==> V2 is best!
min (M - s/v) ==> hV4

min (M - s/v)**2
min (1/M - v/s)**2
```

In [ ]:
(fig,axs) = plt.subplots(2,2, figsize=(7,7), dpi=72*8, sharex=True, sharey=True)

colors = np.array(['c','r','k'])
markers = np.array(['o','s','^'])

for (lbl,ax) in zip([1,2,3,4], axs.flat):
    df = mdls[lbl-1].predict(label=lbl, dataframe=dataframe)
    x = df['vdst_meas']
    y = df['cdst_meas']
    print(np.mean((x-y)**2))
    hem = df['hemi']
    ecc = df['eccentricity']
    mer = df['meridian']
    mrk = markers[
        #(mer == 'upper') + (mer == 'lower')*2]
        (ecc == 5) + 2*(ecc == 10)]
    clr = colors[
        #(hem == 'rh') + (hem == 'lr')*2]
        #(ecc == 5) + 2*(ecc == 10)]
        (mer == 'upper') + (mer == 'lower')*2]
    clr = np.abs(x - y)
    for m in markers:
        ii = (mrk == m)
        ax.scatter(x[ii], y[ii], c=clr[ii], marker=m, s=4, alpha=0.5)
    ax.set_title(f'V{lbl}')
    if lbl > 2:
        ax.set_xlabel('Meas. Crowding Dist. [deg]')
    if lbl == 1 or lbl == 3:
        ax.set_ylabel('Pred. Crowding Dist. [deg]')
    ax.plot([0,8],[0,8], ':', c='0.5', zorder=-1, lw=0.5)

plt.show()

## Crowding Distance Calculations

In [ ]:
# each subject has 1 crowding distance value at each eccentricity
mean_cd = (
    crowding_data
    .groupby(['ID','RadialEccentricity'])
    ['CrowdingDistance']
    .apply(gmean)
    .reset_index())

In [ ]:
cd_list = crowding_data['CrowdingDistance'].tolist()
mean_cd_list = mean_cd['CrowdingDistance'].tolist()

In [ ]:
# create 3 dfs based on the eccentricities in the dataframe:
crowding_eccens = np.unique(crowding_data['RadialEccentricity'])
assert len(crowding_eccens) == 3
(ecc_1, ecc_2, ecc_3) = crowding_eccens

mean_1 = mean_cd[mean_cd['RadialEccentricity'] == ecc_1]
n_1 = len(mean_1)
m_1 = mean_1['CrowdingDistance'].mean()
st_1 = mean_1['CrowdingDistance'].std()

mean_2 = mean_cd[mean_cd['RadialEccentricity'] == ecc_2]
n_2 = len(mean_2)
m_2 = mean_2['CrowdingDistance'].mean()
st_2 = mean_2['CrowdingDistance'].std()

mean_3 = mean_cd[mean_cd['RadialEccentricity'] == ecc_3]
n_3 = len(mean_3)
m_3 = mean_3['CrowdingDistance'].mean()
st_3 = mean_3['CrowdingDistance'].std()

In [ ]:
x_ecc = crowding_data['RadialEccentricity'].tolist()
mean_x_ecc = mean_cd['RadialEccentricity'].tolist()

In [ ]:
# The crowding distance function in terms of eccentricity function described
# by Kurzawski et al. (2023):
Kurzawski2023_cd = cc.crowding.Kurzawski2023_cd

# Fit the b parameter using this function by minimizing log error.
b, _ = curve_fit(cc.crowding.log_Kurzawski2023_cd, x_ecc, np.log10(cd_list), p0=0.15)
b = b[0]
b

In [ ]:
mean_values = [m_1, m_2, m_3]
std_values = [st_1, st_2, st_3]
sem_values = np.array([st_1, st_2, st_3]) / np.sqrt([n_1, n_2, n_3])
eccentricities = [ecc_1, ecc_2, ecc_3]

In [ ]:
# calculate the bouma factor
[val / div for val, div in zip(mean_values, eccentricities)]

### bootstrap on crowding distance fit

In [ ]:
num_bootstrap_samples = 1000
x = np.linspace(0.5,11,1000)
eccentricities = [2.5, 5, 10]

sid_df = crowding_data['ID'].values
x_ecc = np.array(x_ecc)
cd = np.array(cd_list)

bootstrapped = cc.crowding.bootstrap_fit(sid_df, x_ecc, cd, x, num_bootstrap_samples)

# Calculate confidence interval
confidence_interval_cd = np.percentile(bootstrapped, [2.5, 97.5], axis=0)

In [ ]:
plt.figure(figsize=(3.5, 3))

# Fitted value without bootstrap
plt.plot(x, (0.43 + x + 0.06*(x**2)) * b, 'k-', label='Fitted Crowding Distance')

# Plot individual data
plt.plot(mean_x_ecc, mean_cd_list, 'ko', alpha=0.1, label='Individual Crowding Distance')

# Plot error bars
plt.errorbar(eccentricities, mean_values, yerr=std_values, fmt='o', color='red', label='Mean ± Std')
plt.fill_between(x, confidence_interval_cd[0], confidence_interval_cd[1], color='gray', alpha=0.3, label='95% Confidence Interval')

plt.xlabel('Eccentricity (deg)')
plt.ylabel('Crowding distance (deg)')

plt.ylim(bottom=0.1)  # Set lower limit to 0.1 (10^-1)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.show()


## Fit Cortical Magnification

In [ ]:
df = dict(sid=[], h=[], label=[], a=[], b=[], loss=[])
for sid in sids_orig:
    print(sid)
    for h in ['lh','rh']:
        for lbl in [1,2,3,4]:
            try:
                r = cc.cmag.fit_cumarea(sid, h, lbl)
            except Exception as e:
                print(f"  - Skipping: {type(e)}")
                continue
            df['sid'].append(sid)
            df['h'].append(h)
            df['label'].append(lbl)
            df['a'].append(r.x[0])
            df['b'].append(r.x[1])
            df['loss'].append(r.fun)
HH91_params = pd.DataFrame(df)

## check the quality of the fits

In [ ]:
res = {'lh': [[],[],[],[]], 'rh': [[],[],[],[]]}
std_ecc = np.linspace(0.25, 12, 100)
for sid in sids_orig:

    hh91_fits = HH91_params[HH91_params['sid'] == sid]

    for i, lbl in enumerate([1, 2, 3, 4]):
        for j, hemi in enumerate(['lh','rh']):
            hfit_row = hh91_fits[(hh91_fits['label'] == lbl) & (hh91_fits['h'] == hemi)]
            if len(hfit_row) == 0:
                continue

            a = hfit_row['a'].values[0]
            b = hfit_row['b'].values[0]

            (ecc,srf) = cc.cmag.cmag_basics(sid, hemi, lbl)
            ii = np.argsort(ecc)
            ecc = ecc[ii]
            cum_area = np.cumsum(srf[ii])

            cumarea_fit = cc.cmag.HH91_integral(ecc, a, b)
            
            cum_area = np.interp(std_ecc, ecc, cum_area)
            cumarea_fit = np.interp(std_ecc, ecc, cumarea_fit)
            
            diff = cumarea_fit - cum_area
            diff[std_ecc > np.max(ecc)] = np.nan
            res[hemi][lbl - 1].append(diff)

In [ ]:
diffs = {
    h: np.array(dat)
    for (h,dat) in res.items()}

In [ ]:
# mean a,b fits per hemisphere and label
avg_ab = HH91_params.groupby(['label', 'h'])[['a', 'b']].mean()
avg_ab = avg_ab.reset_index()

In [ ]:
(fig, axs) = plt.subplots(4, 2, figsize=(7, 7), dpi=288, sharex=True, sharey=True)

for i, lbl in enumerate([1, 2, 3, 4]):
    for j, hemi in enumerate(['lh', 'rh']):
        ax = axs[i, j]

        hfit_row = avg_ab[(avg_ab['label'] == lbl) & (avg_ab['h'] == hemi)]
        a = hfit_row['a'].values[0]
        b = hfit_row['b'].values[0]

        (ecc, srf) = cc.cmag.cmag_basics(sid, hemi, lbl)
        ii = np.argsort(ecc)
        ecc = ecc[ii]
        cum_area = np.cumsum(srf[ii])

        cumarea_fit = cc.cmag.HH91_integral(ecc, a, b)
        cumarea_fit = np.interp(std_ecc, ecc, cumarea_fit)

        diff_mtx = diffs[hemi][lbl - 1]
        diff_mtx = np.where(np.isfinite(diff_mtx), diff_mtx, np.nan)

        # abs-percentile bounds
        med_low, med_high = cc.cmag.signed_bounds_from_abs_ranking(diff_mtx, 50)
        p95_low, p95_high = cc.cmag.signed_bounds_from_abs_ranking(diff_mtx, 95)

        # scale to cumulative-area units
        y_med_low  = (cumarea_fit + med_low) / 100
        y_med_high = (cumarea_fit + med_high) / 100
        y_p95_low  = (cumarea_fit + p95_low) / 100
        y_p95_high = (cumarea_fit + p95_high) / 100

        ax.plot(std_ecc, cumarea_fit/100, color='blue', label="H&H Model Fit")

        # dark gray: 50% 
        ax.fill_between(std_ecc, y_med_low, y_med_high, color='0.5', alpha=0.7, label='50% of Data')

        # light gray: 95% 
        ax.fill_between(std_ecc, y_p95_low, y_p95_high, color='0.3', alpha=0.3, label='95% of Data')
        
        axs[3,0].set_xlabel('Eccentricity [degree]')
        axs[3,1].set_xlabel('Eccentricity [degree]')
        ax.set_ylabel(r'Surface Area [cm$^2$]')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        if i == 0 and j == 1:
            ax.legend(fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# make a figure for each subject

save_dir = os.path.expanduser('~/for_crowding_figures')
os.makedirs(save_dir, exist_ok=True)

for sid in sids_orig:
    hh91_fits = HH91_params[HH91_params['sid'] == sid]
    fig, axs = plt.subplots(4, 2, figsize=(7, 7), dpi=72*8, sharex=True, sharey=True)

    for i, lbl in enumerate([1, 2, 3, 4]):
        for j, hemi in enumerate(['lh', 'rh']):
            ax = axs[i, j]
            hfit_row = hh91_fits[(hh91_fits['label'] == lbl) & (hh91_fits['h'] == hemi)]
            if len(hfit_row) == 0:
                continue

            a = hfit_row['a'].values[0]
            b = hfit_row['b'].values[0]
            ecc, srf = cc.cmag.cmag_basics(sid, hemi, lbl)
            ii = np.argsort(ecc)
            ecc = ecc[ii]
            cum_area = np.cumsum(srf[ii])
            cumarea_fit = cc.cmag.HH91_integral(ecc, a, b)

            ax.plot(ecc, cumarea_fit / 100, label="H&H Model Fit", color='blue') 
            ax.plot(ecc, cum_area / 100, label="Cumulative Surface Area", color='gray')
            ax.set_ylabel(r'Surface Area [cm$^2$]')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    axs[0, 1].legend()
    axs[3, 0].set_xlabel('Eccentricity [degree]')
    axs[3, 1].set_xlabel('Eccentricity [degree]')
    plt.tight_layout()

    # Save figure
    fig_path = os.path.join(save_dir, f'fits_qc_{sid}.png')
    fig.savefig(fig_path)
    plt.close(fig)  


## bootstrap on C.Mag fits and plot C.Mag against eccentricity

In [ ]:
# mean a,b fits per label
mean_params = HH91_params.groupby('label')[['a', 'b']].mean().reset_index()
np.round(mean_params,2)

In [ ]:
x = np.linspace(0.5, 11, 1000)
cmag_per_label = {}

# fitted Cmag for each area
for _, row in mean_params.iterrows():
    label = row['label']
    a = row['a']
    b = row['b']
    cmag_r = (a / (b + x))**2
    cmag_per_label[label] = cmag_r


In [ ]:
# also calculate std of a,b parameters
np.round(HH91_params.groupby('label')[['a', 'b']].agg(['mean', 'std']).reset_index(),2)

In [ ]:
num_bootstrap_samples = 1000

bootstrap_cmag_per_label = {}

for lbl in [1,2,3,4]:
    df_lbl = HH91_params[HH91_params['label']==lbl]
    a_values = df_lbl['a'].values
    b_values = df_lbl['b'].values

    # Subjects added to keep track
    sids = df_lbl['sid'].values

    boot_curves = []
    n = len(a_values)

    for _ in range(num_bootstrap_samples):
        indices = np.random.choice(n, size=n, replace=True)
        mean_a = np.mean(a_values[indices])
        mean_b = np.mean(b_values[indices])

        # cmag curve
        cmag_boot = (mean_a / (mean_b + x))**2
        boot_curves.append(cmag_boot)

    # shape (1000, 1000)
    boot_curves = np.array(boot_curves)
    bootstrap_cmag_per_label[lbl] = boot_curves


In [ ]:
bootstrapped_v1 = bootstrap_cmag_per_label[1]
bootstrapped_v2 = bootstrap_cmag_per_label[2]
bootstrapped_v3 = bootstrap_cmag_per_label[3]
bootstrapped_v4 = bootstrap_cmag_per_label[4]

In [ ]:
confidence_interval_v1 = np.percentile(bootstrapped_v1, [2.5, 97.5], axis=0)
confidence_interval_v2 = np.percentile(bootstrapped_v2, [2.5, 97.5], axis=0)
confidence_interval_v3 = np.percentile(bootstrapped_v3, [2.5, 97.5], axis=0)
confidence_interval_v4 = np.percentile(bootstrapped_v4, [2.5, 97.5], axis=0)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5, 3.5), dpi=72*8)

# Plotting the fitted lines for each visual area
ax.plot(x, np.sqrt(cmag_per_label[1]/2), 'k', label='V1 fitted line')
ax.plot(x, np.sqrt(cmag_per_label[2]/2), 'r', label='V2 fitted line')
ax.plot(x, np.sqrt(cmag_per_label[3]/2), 'm', label='V3 fitted line')
ax.plot(x, np.sqrt(cmag_per_label[4]/2), 'cyan', label='hV4 fitted line')

# Plotting the confidence intervals for each visual area
ax.fill_between(x, 
                 np.sqrt(confidence_interval_v1[0]/2),
                 np.sqrt(confidence_interval_v1[1]/2),
                 color='k', alpha=0.3)
ax.fill_between(x, 
                 np.sqrt(confidence_interval_v2[0]/2),
                 np.sqrt(confidence_interval_v2[1]/2),
                 color='r', alpha=0.3)
ax.fill_between(x, 
                 np.sqrt(confidence_interval_v3[0]/2),
                 np.sqrt(confidence_interval_v3[1]/2),
                 color='m', alpha=0.3)
ax.fill_between(x, 
                 np.sqrt(confidence_interval_v4[0]/2),
                 np.sqrt(confidence_interval_v4[1]/2),
                 color='cyan', alpha=0.3)

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Eccentricity [deg]")
ax.set_ylabel("Radial Cortical Magnification [mm/deg]")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend()
plt.show()

## cortical crowding distance (ccd) and coefficent of variation for ccd

In [ ]:
bootstrapped_ccd_1 = []
bootstrapped_ccd_2 = []
bootstrapped_ccd_3 = []
bootstrapped_ccd_4 = []

# Iterate through 1000 bootstrapped samples
for i in range(1000):
    ccd_v1 = bootstrapped[i] * np.sqrt(bootstrapped_v1[i] / 2)
    bootstrapped_ccd_1.append(ccd_v1)
    
    ccd_v2 = bootstrapped[i] * np.sqrt(bootstrapped_v2[i] / 2)
    bootstrapped_ccd_2.append(ccd_v2)
    
    ccd_v3 = bootstrapped[i] * np.sqrt(bootstrapped_v3[i] / 2)
    bootstrapped_ccd_3.append(ccd_v3)
    
    ccd_v4 = bootstrapped[i] * np.sqrt(bootstrapped_v4[i] / 2)
    bootstrapped_ccd_4.append(ccd_v4)

bootstrapped_ccd_1 = np.array(bootstrapped_ccd_1)
bootstrapped_ccd_2 = np.array(bootstrapped_ccd_2)
bootstrapped_ccd_3 = np.array(bootstrapped_ccd_3)
bootstrapped_ccd_4 = np.array(bootstrapped_ccd_4)

# Calculate the mean of bootstrapped CCD values for each visual area
ccd1 = np.mean(bootstrapped_ccd_1, axis=0)
ccd2 = np.mean(bootstrapped_ccd_2, axis=0)
ccd3 = np.mean(bootstrapped_ccd_3, axis=0)
ccd4 = np.mean(bootstrapped_ccd_4, axis=0)

# Calculate the confidence interval for bootstrapped CCD values for each visual area
confidence_interval_ccd_1 = np.percentile(bootstrapped_ccd_1,  [16, 84], axis=0)
confidence_interval_ccd_2 = np.percentile(bootstrapped_ccd_2,  [16, 84], axis=0)
confidence_interval_ccd_3 = np.percentile(bootstrapped_ccd_3,  [16, 84], axis=0)
confidence_interval_ccd_4 = np.percentile(bootstrapped_ccd_4,  [16, 84], axis=0)

In [ ]:
# Calculate the coefficient of variation for ccd_1
mean_ccd_1 = np.mean(ccd1)
std_ccd_1 = np.std(ccd1)
cv_ccd_1 = std_ccd_1 / mean_ccd_1
rounded_cv_ccd_1 = round(cv_ccd_1, 3)

print("Coefficient of Variation (CCD 1):", rounded_cv_ccd_1)

# Calculate the coefficient of variation for ccd_2
mean_ccd_2 = np.mean(ccd2)
std_ccd_2 = np.std(ccd2)
cv_ccd_2 = std_ccd_2 / mean_ccd_2
rounded_cv_ccd_2 = round(cv_ccd_2, 3)
print("Coefficient of Variation (CCD 2):", rounded_cv_ccd_2)

# Calculate the coefficient of variation for ccd_3
mean_ccd_3 = np.mean(ccd3)
std_ccd_3 = np.std(ccd3)
cv_ccd_3 = std_ccd_3 / mean_ccd_3
rounded_cv_ccd_3 = round(cv_ccd_3, 3)
print("Coefficient of Variation (CCD 3):", rounded_cv_ccd_3)

# Calculate the coefficient of variation for ccd_4
mean_ccd_4 = np.mean(ccd4)
std_ccd_4 = np.std(ccd4)
cv_ccd_4 = std_ccd_4 / mean_ccd_4
rounded_cv_ccd_4 = round(cv_ccd_4, 3)
print("Coefficient of Variation (CCD 4):", rounded_cv_ccd_4)

In [ ]:
bootstrapped_cv_1 = cc.corticalcrowding.bootstrap_cv(ccd1)
ci_1 = np.percentile(bootstrapped_cv_1, [2.5, 97.5])

bootstrapped_cv_2 = cc.corticalcrowding.bootstrap_cv(ccd2)
ci_2 = np.percentile(bootstrapped_cv_2, [2.5, 97.5])

bootstrapped_cv_3 = cc.corticalcrowding.bootstrap_cv(ccd3)
ci_3 = np.percentile(bootstrapped_cv_3, [2.5, 97.5])

bootstrapped_cv_4 = cc.corticalcrowding.bootstrap_cv(ccd4)
ci_4 = np.percentile(bootstrapped_cv_4, [2.5, 97.5])

In [ ]:
cv_ccd_1 = bootstrapped_cv_1.mean()
cv_ccd_2 = bootstrapped_cv_2.mean()
cv_ccd_3 = bootstrapped_cv_3.mean()
cv_ccd_4 = bootstrapped_cv_4.mean()
print([cv_ccd_1, cv_ccd_2, cv_ccd_3, cv_ccd_4]) 

In [ ]:
print(np.round(ci_1, 2))
print(np.round(ci_2, 2))
print(np.round(ci_3, 2))
print(np.round(ci_4, 2))

In [ ]:
cv_ccd_values = [cv_ccd_1, cv_ccd_2, cv_ccd_3, cv_ccd_4]
ccd_labels = ['V1', 'V2', 'V3', 'V4']
cv_ci_list = [(ci_1[0], ci_1[1]), (ci_2[0], ci_2[1]), (ci_3[0], ci_3[1]),(ci_4[0], ci_4[1])]

lower_bound = [ci[0] for ci in cv_ci_list]
upper_bound = [ci[1] for ci in cv_ci_list]

yerr = [[cv_ccd_values[i] - lower_bound[i] for i in range(len(cv_ccd_values))],
        [upper_bound[i] - cv_ccd_values[i] for i in range(len(cv_ccd_values))]]

plt.figure(figsize=(3.5, 3))
plt.bar(ccd_labels, cv_ccd_values, yerr=yerr, capsize=5, color=['grey', 'red', 'magenta', 'cyan'])

plt.xlabel('Visual Areas')
plt.ylabel('Coefficient of Variation')
plt.title('Coefficient of Variation for Cortical Crowding Distance')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()


In [ ]:
# plot the mean cortical crowding distance of each area
plt.figure(figsize=(3.5, 3))
plt.plot(x, ccd1, label='Cortical crowding distance in V1', color='black')
plt.plot(x, ccd2, label='Cortical crowding distance in V2', color='red')
plt.plot(x, ccd3, label='Cortical crowding distance in V3', color='magenta')
plt.plot(x, ccd4, label='Cortical crowding distance in hV4', color='cyan')

plt.fill_between(x, confidence_interval_ccd_1[0], confidence_interval_ccd_1[1], color='black', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_2[0], confidence_interval_ccd_2[1], color='red', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_3[0], confidence_interval_ccd_3[1], color='magenta', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_4[0], confidence_interval_ccd_4[1], color='cyan', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xlabel('Eccentricity (deg)')
plt.ylabel('Cortical Crowding Distance (mm)')
plt.show()

In [ ]:
# normalize the cortical crowding distance
plt.figure(figsize=(3.5, 3))
plt.plot(x, ccd1/ccd1[0], color='black')
plt.plot(x, ccd2/ccd2[0], color='red')
plt.plot(x, ccd3/ccd3[0], color='magenta')
plt.plot(x, ccd4/ccd4[0], color='cyan')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.xlabel('Eccentricity (deg)')
plt.ylabel('Normalized Cortical C.D. (mm)')
#plt.xticks([2.5, 5, 10])
plt.show()

## linear regression: predict crowding distance based on vmag_fit

In [ ]:
# average parameter values across hemispheres for each subject and label
HH91_params_bi = HH91_params.groupby(
    ['sid', 'label'], as_index=False
    ).agg({
    'a': 'mean',
    'b': 'mean',
    'loss': 'mean'
})

In [ ]:
df_mean = HH91_params_bi.merge(
    pd.DataFrame(dict(RadialEccentricity=[2.5, 5.0, 10.0])),
    how='cross')

a = df_mean['a']
b = df_mean['b']
ecc = df_mean['RadialEccentricity']
# calculate cmag based on a,b params from HH91_params_bi
df_mean['cmag_fit'] = (a / (ecc + b))**2
# add 1d visual magnification
df_mean['vmag1d_fit'] = np.sqrt(1 / df_mean['cmag_fit'])
df_mean['cmag_rad_fit'] = np.sqrt(df_mean['cmag_fit'] / 2)
df_mean['vmag_rad_fit'] = 1.0 / df_mean['cmag_rad_fit']

In [ ]:
# rename columns of crowding distance df
mean_cd = mean_cd.rename(columns={"ID": "sid"})

In [ ]:
df_mean = df_mean.merge(mean_cd, on=['sid', 'RadialEccentricity'])

In [ ]:
df_mean #15 subjects with both crowding distance and fMRI data

In [ ]:
(fig,axs) = plt.subplots(2, 2, figsize=(7,7), dpi=72*8, sharex=True, sharey=True)

cm = mpl.cm.jet

for (lbl,ax) in zip([1,2,3,4], axs.flat):
    ax.set_title(f'V{lbl}')
    df = df_mean[df_mean['label'] == lbl]
    sids = np.unique(df['sid'].values)
    for (ii,sid) in enumerate(sids):
        df_sid = df[df['sid'] == sid]
        x = df_sid['vmag_rad_fit']
        y = df_sid['CrowdingDistance']
        ax.plot(x, y, '.-', c=cm(ii / (len(sids) - 1)), alpha=0.5)

plt.show()

In [ ]:
res = {1: [], 2: [], 3: [], 4: []}

for subject, subject_data in df_mean.groupby(['sid']):
    if (subject_data['b'] > 5).any():
        continue
    for lbl in [1, 2, 3, 4]:
        ssdf = subject_data[subject_data['label'] == lbl]
        #x = ssdf['vmag1d_fit'].values
        x = ssdf['vmag_rad_fit'].values
        y = ssdf['CrowdingDistance'].values
        rss, coef = cc.regression.fit_and_evaluate(x, y)
        res[lbl].append(rss)

mean_rss = [np.mean(res[l]) for l in [1, 2, 3, 4]]
std_rss  = [np.std(res[l])  for l in [1, 2, 3, 4]]
n_subjects = 15
sem_rss  = np.array(std_rss) / np.sqrt(n_subjects)

print(mean_rss)
print(std_rss)


In [ ]:
labels = ['V1', 'V2', 'V3', 'V4']
plt.figure(figsize=(8, 6))
plt.bar(labels, mean_rss, yerr=sem_rss)
plt.ylabel("RSS (Residual Sum of Squares)")
plt.title("Mean RSS with Standard Error Across Visual Areas")
plt.show()

# previous code used in VSS24

### bootstrap crowding distance

In [ ]:
def func_cd(x, b):
    return np.log10((0.43 + x + 0.06*(x**2)) * b)

# the number of bootstrap samples
num_bootstrap_samples = 1000
x = np.linspace(0.5,10,1000)
eccentricities = [2.5, 5, 10]

# sid_df.shape=(480,)
sid_df = df['Observer'].values
x_ecc = np.array(x_ecc)
cd = np.array(cd_list)

def bootstrap_fit(sids, xdata, ydata, x):
    # unique_sids : 20 numbers
    unique_sids = np.unique(sids)
    bootstrapped_parameters = []
    for _ in range(num_bootstrap_samples):
        # each bootstrap, sample 20 subjects with replacement
        indices = np.random.choice(unique_sids, size=len(unique_sids), replace=True)
        indices = [np.where(sids == sid)[0] for sid in indices]
        indices = [k for ak in indices for k in ak]
        # 20 by 24 = 480, 480 x values and y values each
        x_boot = xdata[indices]
        y_boot = ydata[indices]
        # Fit the curve to the bootstrapped sample
        b, _ = curve_fit(func_cd, x_boot, np.log10(y_boot), p0=0.15)
        y = (0.43 + x + 0.06*(x**2)) * b
        bootstrapped_parameters.append(y) 
    return bootstrapped_parameters

bootstrapped = bootstrap_fit(sid_df, x_ecc, cd, x)

# Calculate confidence interval
confidence_interval_cd = np.percentile(bootstrapped, [2.5, 97.5], axis=0)

In [ ]:
x = np.linspace(0.5, 10, 1000)
# Fitted value without bootstrap
plt.plot(x, (0.43 + x + 0.06*(x**2)) * b, 'k-', label='Fitted Crowding Distance')

# Plot individual data
plt.plot(mean_x_ecc, mean_cd_list, 'ko', alpha=0.1, label='Individual Crowding Distance')

# Plot error bars
plt.errorbar(eccentricities, mean_values, yerr=std_values, fmt='o', color='red', label='Mean ± Std')
plt.fill_between(x, confidence_interval_cd[0], confidence_interval_cd[1], color='gray', alpha=0.3, label='95% Confidence Interval')

plt.xlabel('Eccentricity (deg)')
plt.ylabel('Crowding distance (deg)')
plt.yscale('log')
plt.ylim(bottom=0.1)  # Set lower limit to 0.1 (10^-1)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

### bootstrap C.Mag

In [ ]:
# calculate cmag
all_cmag_v1 = []
all_cmag_v2 = []
all_cmag_v3 = []
all_cmag_v4 = []
all_eccen_v1 = []
all_eccen_v2 = []
all_eccen_v3 = []
all_eccen_v4 = []
eccen = np.linspace(1, 11, 1000)
subjects_added = []  
all_mask = ('variance_explained', 0.04, 1)

for sid in sids:
    try:
        sub = load_subject(sid)

        # Calculate cmag for the subject for V1
        v1_mask = {'and': [('visual_area', 1), all_mask]}
        eccen_v1, cmag_v1 = ring_cmag(sub, eccen=None, mask=v1_mask)
        all_eccen_v1.append(eccen_v1)
        all_cmag_v1.append(cmag_v1)

        # Calculate cmag for the subject for V2
        v2_mask = {'and': [('visual_area', 2), all_mask]}
        eccen_v2, cmag_v2 = ring_cmag(sub, eccen=None, mask=v2_mask)
        all_eccen_v2.append(eccen_v2)
        all_cmag_v2.append(cmag_v2)

        # Calculate cmag for the subject for V3
        v3_mask = {'and': [('visual_area', 3), all_mask]}
        eccen_v3, cmag_v3 = ring_cmag(sub, eccen=None, mask=v3_mask)
        all_eccen_v3.append(eccen_v3)
        all_cmag_v3.append(cmag_v3)
        
        # Calculate cmag for the subject for V4
        v4_mask = {'and': [('visual_area', 4), all_mask]}
        eccen_v4, cmag_v4 = ring_cmag(sub, eccen=None, mask=v4_mask)
        all_eccen_v4.append(eccen_v4)
        all_cmag_v4.append(cmag_v4)
        
        subjects_added.append(sid)  # Add subject to the list of subjects added
    
    except Exception as e:
        print(f"Error calculating cmag for subject {sid}: {e}")


# Convert lists to arrays
# all_cmag_v1 = np.array(all_cmag_v1)
# all_cmag_v2 = np.array(all_cmag_v2)
# all_cmag_v3 = np.array(all_cmag_v3)
# all_cmag_v4: len=35, each array has diff shape
all_flatcmag_v4 = np.concatenate(all_cmag_v4)
all_flateccen_v4 = np.concatenate(all_eccen_v4)

In [ ]:
eccen = np.linspace(1,11, 1000)

def func(x, a, b):
    return (a / (b + x))**2
# use all_cmag_v1 contains cmag for v1 for 29 subjects
subjects_added = np.array(subjects_added)
x_data = np.array(eccen)
cmag_v1 = np.array(all_cmag_v1)
cmag_v2 = np.array(all_cmag_v2)
cmag_v3 = np.array(all_cmag_v3)
cmag_v4 = np.array(all_cmag_v4)

def bootstrap_fit_cmag(sids, xdata, ydata, x, p0):
    unique_sids = np.unique(sids)
    bootstrapped_parameters = []
    for _ in range(num_bootstrap_samples):
        # Sample subjects
        indices = np.random.choice(unique_sids, size=len(unique_sids), replace=True)
        indices = [np.where(sids == sid)[0] for sid in indices]
        indices = [k for ak in indices for k in ak]
        x_boot = xdata[indices]
        y_boot = ydata[indices]
        # Fit the curve to the bootstrapped sample
        popt, _ = curve_fit(func, x_boot.flatten(), y_boot.flatten(),p0=p0)
        # store the function value
        y = (popt[0] / (popt[1] + x))**2
        bootstrapped_parameters.append(y) 
    return bootstrapped_parameters

all_eccen = np.array([x_data]*len(subjects_added))

bootstrapped_v1 = bootstrap_fit_cmag(subjects_added, all_eccen, cmag_v1, eccen, p0=[17.3, 0.75])
bootstrapped_v2 = bootstrap_fit_cmag(subjects_added, all_eccen, cmag_v2, eccen, p0=[17.3, 0.75])
bootstrapped_v3 = bootstrap_fit_cmag(subjects_added, all_eccen, cmag_v3, eccen, p0=[17.3, 0.75])
bootstrapped_v4 = bootstrap_fit_cmag(subjects_added, all_eccen, cmag_v4, eccen, p0=[17.3, 0.75])

# Calculate confidence interval
confidence_interval_v1 = np.percentile(bootstrapped_v1, [2.5, 97.5], axis=0)
confidence_interval_v2 = np.percentile(bootstrapped_v2, [2.5, 97.5], axis=0)
confidence_interval_v3 = np.percentile(bootstrapped_v3, [2.5, 97.5], axis=0)
confidence_interval_v4 = np.percentile(bootstrapped_v4, [2.5, 97.5], axis=0)

In [ ]:
fig, ax = plt.subplots()

# Plotting the average data for each visual area
ax.plot(eccen, np.sqrt(average_cmag_v1/2), 'k:', label='V1')
ax.plot(eccen, np.sqrt(average_cmag_v2/2), 'r:', label='V2')
ax.plot(eccen, np.sqrt(average_cmag_v3/2), 'm:', label='V3')
ax.plot(eccen, np.sqrt(average_cmag_v4/2), 'g:', label='hV4')

# Plotting the fitted lines for each visual area
ax.plot(eccen, np.sqrt((popt1[0]/(eccen+popt1[1]))**2/2), 'k', label='V1 fitted line')
ax.plot(eccen, np.sqrt((popt2[0]/(eccen+popt2[1]))**2/2), 'r', label='V2 fitted line')
ax.plot(eccen, np.sqrt((popt3[0]/(eccen+popt3[1]))**2/2), 'm', label='V3 fitted line')
ax.plot(eccen, np.sqrt((popt4[0]/(eccen+popt4[1]))**2/2), 'g', label='hV4 fitted line')

# Plotting the confidence intervals for each visual area
ax.fill_between(eccen, 
                 np.sqrt(confidence_interval_v1[0]/2),
                 np.sqrt(confidence_interval_v1[1]/2),
                 color='k', alpha=0.3)
ax.fill_between(eccen, 
                 np.sqrt(confidence_interval_v2[0]/2),
                 np.sqrt(confidence_interval_v2[1]/2),
                 color='r', alpha=0.3)
ax.fill_between(eccen, 
                 np.sqrt(confidence_interval_v3[0]/2),
                 np.sqrt(confidence_interval_v3[1]/2),
                 color='m', alpha=0.3)
ax.fill_between(eccen, 
                 np.sqrt(confidence_interval_v4[0]/2),
                 np.sqrt(confidence_interval_v4[1]/2),
                 color='g', alpha=0.3)

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Eccentricity (deg)")
ax.set_ylabel("Radial Cortical Magnification (mm/deg)")
ax.set_title("Average Radial Cortical Magnification for V1, V2, V3, hV4")
ax.legend()
plt.show()

### using bootstrapped fits to get cortical crowding distance

In [ ]:
# lists to store bootstrapped CCD values for each visual area
bootstrapped_ccd_1 = []
bootstrapped_ccd_2 = []
bootstrapped_ccd_3 = []
bootstrapped_ccd_4 = []

# "bootstrapped" here refers to crowding distance result
for i in range(len(bootstrapped)):
    # Calculate bootstrapped CCD for visual area 1
    ccd_v1 = bootstrapped[i] * np.sqrt(bootstrapped_v1[i] / 2)
    bootstrapped_ccd_1.append(ccd_v1)
    
    # Calculate bootstrapped CCD for visual area 2
    ccd_v2 = bootstrapped[i] * np.sqrt(bootstrapped_v2[i] / 2)
    bootstrapped_ccd_2.append(ccd_v2)
    
    # Calculate bootstrapped CCD for visual area 3
    ccd_v3 = bootstrapped[i] * np.sqrt(bootstrapped_v3[i] / 2)
    bootstrapped_ccd_3.append(ccd_v3)
    
    # Calculate bootstrapped CCD for visual area 4
    ccd_v4 = bootstrapped[i] * np.sqrt(bootstrapped_v4[i] / 2)
    bootstrapped_ccd_4.append(ccd_v4)

# Convert lists to arrays
bootstrapped_ccd_1 = np.array(bootstrapped_ccd_1)
bootstrapped_ccd_2 = np.array(bootstrapped_ccd_2)
bootstrapped_ccd_3 = np.array(bootstrapped_ccd_3)
bootstrapped_ccd_4 = np.array(bootstrapped_ccd_4)

# Calculate the mean of bootstrapped CCD values for each visual area
xx1 = np.mean(bootstrapped_ccd_1, axis=0)
xx2 = np.mean(bootstrapped_ccd_2, axis=0)
xx3 = np.mean(bootstrapped_ccd_3, axis=0)
xx4 = np.mean(bootstrapped_ccd_4, axis=0)

# Calculate the confidence interval for bootstrapped CCD values for each visual area
confidence_interval_ccd_1 = np.percentile(bootstrapped_ccd_1,  [16, 84], axis=0)
confidence_interval_ccd_2 = np.percentile(bootstrapped_ccd_2,  [16, 84], axis=0)
confidence_interval_ccd_3 = np.percentile(bootstrapped_ccd_3,  [16, 84], axis=0)
confidence_interval_ccd_4 = np.percentile(bootstrapped_ccd_4,  [16, 84], axis=0)

### coefficient of variation for cortical crowding distance at each area

In [ ]:
# Calculate the coefficient of variation for ccd_1
mean_ccd_1 = np.mean(xx1)
std_ccd_1 = np.std(xx1)
cv_ccd_1 = std_ccd_1 / mean_ccd_1
rounded_cv_ccd_1 = round(cv_ccd_1, 2)

print("Coefficient of Variation (CCD 1):", rounded_cv_ccd_1)

# Calculate the coefficient of variation for ccd_2
mean_ccd_2 = np.mean(xx2)
std_ccd_2 = np.std(xx2)
cv_ccd_2 = std_ccd_2 / mean_ccd_2
rounded_cv_ccd_2 = round(cv_ccd_2, 2)
print("Coefficient of Variation (CCD 2):", rounded_cv_ccd_2)

# Calculate the coefficient of variation for ccd_3
mean_ccd_3 = np.mean(xx3)
std_ccd_3 = np.std(xx3)
cv_ccd_3 = std_ccd_3 / mean_ccd_3
rounded_cv_ccd_3 = round(cv_ccd_3, 2)
print("Coefficient of Variation (CCD 3):", rounded_cv_ccd_3)

# Calculate the coefficient of variation for ccd_4
mean_ccd_4 = np.mean(xx4)
std_ccd_4 = np.std(xx4)
cv_ccd_4 = std_ccd_4 / mean_ccd_4
rounded_cv_ccd_4 = round(cv_ccd_4, 2)
print("Coefficient of Variation (CCD 4):", rounded_cv_ccd_4)

In [ ]:
def bootstrap_cv(data, num_samples):
    """
    Perform bootstrapping to compute the coefficient of variation (CV).

    Parameters:
        data (numpy.ndarray)
        num_samples (int): Number of bootstrap samples to generate.

    Returns:
        numpy.ndarray: Array containing the bootstrapped CV values.
    """
    bootstrapped_cv = []
    n = len(data)
    for _ in range(num_samples):
        sample_indices = np.random.choice(range(n), size=n, replace=True)
        bootstrapped_sample = data[sample_indices]
        mean_sample = np.mean(bootstrapped_sample)
        std_sample = np.std(bootstrapped_sample)
        cv_sample = std_sample / mean_sample
        bootstrapped_cv.append(cv_sample)
    return np.array(bootstrapped_cv)

# Perform bootstrap on CCD 1
bootstrapped_cv_1 = bootstrap_cv(xx1, num_samples=1000)
ci_1 = np.percentile(bootstrapped_cv_1, [2.5, 97.5])

# Perform bootstrap on CCD 2
bootstrapped_cv_2 = bootstrap_cv(xx2, num_samples=1000)
ci_2 = np.percentile(bootstrapped_cv_2, [2.5, 97.5])

# Perform bootstrap on CCD 3
bootstrapped_cv_3 = bootstrap_cv(xx3, num_samples=1000)
ci_3 = np.percentile(bootstrapped_cv_3, [2.5, 97.5])

# Perform bootstrap on CCD 4
bootstrapped_cv_4 = bootstrap_cv(xx4, num_samples=1000)
ci_4 = np.percentile(bootstrapped_cv_4, [2.5, 97.5])

In [ ]:
cv_ccd_1 = bootstrapped_cv_1.mean()
cv_ccd_2 = bootstrapped_cv_2.mean()
cv_ccd_3 = bootstrapped_cv_3.mean()
cv_ccd_4 = bootstrapped_cv_4.mean()

In [ ]:
# a list of mean CV in each area
cv_ccd_values = [cv_ccd_1, cv_ccd_2, cv_ccd_3, cv_ccd_4]
ccd_labels = ['V1', 'V2', 'V3', 'V4']

# a list of CI in each area
cv_ci_list = [(ci_1[0], ci_1[1]), (ci_2[0], ci_2[1]), (ci_3[0], ci_3[1]),(ci_4[0], ci_4[1])]

# lower and upper bounds of CI
lower_bound = [ci[0] for ci in cv_ci_list]
upper_bound = [ci[1] for ci in cv_ci_list]

yerr = [[cv_ccd_values[i] - lower_bound[i] for i in range(len(cv_ccd_values))],
        [upper_bound[i] - cv_ccd_values[i] for i in range(len(cv_ccd_values))]]

# bar plot with error bars
plt.bar(ccd_labels, cv_ccd_values, yerr=yerr, capsize=5, color=['grey', 'red', 'magenta', 'green'])

plt.xlabel('Visual Areas')
plt.ylabel('Coefficient of Variation')
plt.title('Coefficient of Variation for Cortical Crowding Distance')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

In [ ]:
# Plot cortical crowding distance vs eccen
x = np.linspace(0.5, 10, 1000)
plt.plot(x, xx1, label='Cortical crowding distance in V1', color='black')
plt.plot(x, xx2, label='Cortical crowding distance in V2', color='red')
plt.plot(x, xx3, label='Cortical crowding distance in V3', color='magenta')
plt.plot(x, xx4, label='Cortical crowding distance in hV4', color='green')

# Confidence intervals
plt.fill_between(x, confidence_interval_ccd_1[0], confidence_interval_ccd_1[1], color='black', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_2[0], confidence_interval_ccd_2[1], color='red', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_3[0], confidence_interval_ccd_3[1], color='magenta', alpha=0.3)
plt.fill_between(x, confidence_interval_ccd_4[0], confidence_interval_ccd_4[1], color='green', alpha=0.3)

plt.xlabel('Eccentricity (deg)')
plt.ylabel('Cortical Crowding Distance (mm)')
#plt.legend()
plt.show()

In [ ]:
# to check quality of fits
# (fig, axs) = plt.subplots(4, 2, figsize=(7, 7), dpi=288, sharex=True, sharey=True)

# for i, lbl in enumerate([1, 2, 3, 4]):
#     for j, hemi in enumerate(['lh', 'rh']):
#         ax = axs[i, j]
#         hfit_row = avg_ab[(avg_ab['label'] == lbl) & (avg_ab['h'] == hemi)]
#         a = hfit_row['a'].values[0]
#         b = hfit_row['b'].values[0]

#         (ecc, srf) = cmag_basics(sid, hemi, lbl)
#         ii = np.argsort(ecc)
#         ecc = ecc[ii]
#         cum_area = np.cumsum(srf[ii])

#         cumarea_fit = cc.cmag.HH91_integral(ecc, a, b)
#         cumarea_fit = np.interp(std_ecc, ecc, cumarea_fit)

#         diff_mtx = diffs[hemi][lbl - 1]
#         ii = np.isfinite(diff_mtx)
#         diff_mtx[ii] = np.abs(diff_mtx[ii])
#         diff_med = np.nanmedian(diff_mtx, axis=0)
#         diff_95 = np.nanpercentile(diff_mtx, 95, axis=0)

#         ax.plot(std_ecc, cumarea_fit/100, label="H&H Model Fit", color='blue') 
#         ax.fill_between(std_ecc, (cumarea_fit - diff_med)/100, (cumarea_fit + diff_med)/100, color='0.5', alpha=0.7, label='Median Error')
#         ax.fill_between(std_ecc, (cumarea_fit - diff_95)/100, (cumarea_fit + diff_95)/100, color='0.3', alpha=0.3, label='95% Error')

#         axs[3,0].set_xlabel('Eccentricity [degree]')
#         axs[3,1].set_xlabel('Eccentricity [degree]')
#         ax.set_ylabel(r'Surface Area [cm$^2$]')
#         ax.spines['top'].set_visible(False)
#         ax.spines['right'].set_visible(False)

#         axs[0, 1].legend(fontsize=10)

# plt.tight_layout()
# plt.show()
